In [1]:
import os

os.chdir('/home/enis/projects/nna/src/nna/exp/megan/run-1/')


In [2]:
# import run
# import nna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchaudio
torchaudio.set_audio_backend("sox_io")

import numpy as np

/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
best_solution_per_taxo_w_loc_name={'1.0.0': [['44', '46', '17', '14'],
  ['11', '34', '27'],
  ['31', '50', '18', '12', '30', '39', '48', '45']],
 '3.0.0': [['40', '20', '14', '17', '13', '36', '25', '33'],
  ['18', '38', '39'],
  ['32', '45']],
 'X.X.X': [['45', '14', '27', '25', '34', '46', '29', '18', '38'],
  ['36'],
  ['32', '20', '21']],
 '1.1.10': [['49', '48', '19', '16', '22', '37', '29', '25', '31', '27'],
  ['46', '20', '11', '33', '24'],
  ['17', '21', '39', '30', '38', '18', '47', '50', '14']],
 '1.1.0': [['12',
   '37',
   '11',
   '22',
   '18',
   '44',
   '29',
   '46',
   '13',
   '34',
   '25',
   '24',
   '17',
   '40',
   '31',
   '27',
   '14'],
  ['19', '16', '39', '30', '38', '41'],
  ['50', '20', '47', '49', '48', '21', '15', '36']],
 '1.3.0': [['21', '40', '32', '39', '41', '44'],
  ['50', '20', '38', '11'],
  ['24', '19', '27', '14']],
 '1.1.8': [['20', '16', '11', '37', '15', '25', '31'],
  ['21', '49', '38'],
  ['22', '40']],
 '1.1.7': [['16', '30', '15', '46', '27'],
  ['49', '25', '24'],
  ['20', '39', '48', '19', '38', '22', '29', '11', '40']],
 '0.2.0': [['17'], ['24', '27'], ['19', '15']],
 '0.0.0': [['29'], ['24'], ['12', '27']],
 '1.1.3': [['46', '25'], ['24'], ['27', '22', '38']],
 '2.1.0': [['34', '33'], ['36'], ['46']],
 '0.1.0': [['29'], ['25'], ['24']]}
error_because_small = ['1.2.4',
'1.1.5',
'1.2.0',
'0.4.0']


In [4]:
import runconfigs
import wandb
from ignite.metrics import Accuracy, Loss
from ignite.contrib.metrics import ROC_AUC

In [5]:
# from nna.exp import augmentations,
from nna.exp import modelArchs,runutils

In [6]:
# wandb.init(config=runconfigs.default_config, project=runconfigs.PROJECT_NAME)
# config = wandb.config
config = runconfigs.default_config
# wandb.config.update(args) # adds all of the arguments as config variables

params = {
    'batch_size': config['batch_size'],
    'shuffle': True,
    'num_workers': 0
}


In [7]:
device = torch.device(
    f"cuda:{config['device']}" if torch.cuda.is_available() else "cpu")


In [8]:
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split

# labelsbyhumanpath = Path('/scratch/enis/data/nna/labeling/results/')
# sourcePath = Path("/scratch/enis/data/nna/labeling/splits/")


In [32]:
CATEGORY_COUNT = 2


In [10]:
from collections import Counter


In [11]:
 # RAW DATA
def load_raw_data(sample_count,CATEGORY_COUNT):
    sample_count = 1200
    X = np.empty((sample_count,480000),dtype=np.float32)
    y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
    for i,y in enumerate(y_true):
        X[i,:] = y
    X = np.interp(X, (X.min(), X.max()), (-32768 , 32767))
    n_values = np.max(y_true) + 1
    y_true = np.eye(n_values)[y_true]
    X_train, X_test, y_train, y_test = train_test_split(
                    X, y_true, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
                    X_train, y_train, test_size=0.25,random_state=42)
    return X_train,X_test,X_val,y_train,y_test,y_val

def mock_raw_data(sample_count,CATEGORY_COUNT):
#     sample_count = 30
    X = np.empty((sample_count*CATEGORY_COUNT,480000),dtype=np.float32)
    # y_true = np.random.randint(0,CATEGORY_COUNT,(sample_count))
    y_true=[]
    for i in range(CATEGORY_COUNT):
        y_true.extend([i]*sample_count)
    print(len(y_true))
    for i,y in enumerate(y_true):
        X[i,:] = y
    X = np.interp(X, (X.min(), X.max()), (0 , 3400))
#     X = np.interp(X, (X.min(), X.max()), (-32768 , 32767))
    n_values = np.max(y_true) + 1
    y_true = np.eye(n_values)[y_true]
    X_train, X_test, y_train, y_test = train_test_split(
                    X, y_true, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
                    X_train, y_train, test_size=0.25,random_state=42)
#     print(y_train)
    return X_train,X_test,X_val,y_train,y_test,y_val
    


In [12]:
## Load real data rather than mock 
    # MVP1: delete parts longer than 10 seconds
import run
audio_dataset,_ = run.prepare_dataset()

output_file_path = '/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/files_as_np_filtered_v3_int16.pkl'
audio_dataset.load_audio_files(output_file_path)
audio_dataset.pick_channel_by_clipping()


/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/meganLabeledFiles_wlenV1.txt
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite
4 files are missing corresponding to excell entries
'-> 5 number of samples are DELETED due to ignore_files and missing_audio_files'
-> 415 samples DELETED because they are not in the excell

-> 0 samples DELETED because they do not have the taxo info coming from excell

-> classes that do not have enough data:
[REMOVED!]
['other-mammal'] 0.0
['other-silence'] 20.0
['unknown-sound'] 2.0
['seabirds'] 1.0
['canids'] 1.0
['other-flare'] 11.0
['other-rain'] 20.0

-> classes that have enough data:
['other-biophony'] 56.0
['other-insect'] 140.0
['other-bird'] 661.0
['songbirds'] 392.0
['duck-goose-swan'] 183.0
['grouse-ptarmigan'] 59.0
['other-anthrophony'] 66.0
['other-aircraft'] 107.0
['loons'] 29.0
['other-car'] 37.0
('-> 102 number of samples are deleted because their taxonomy category does '
 'not have enough data.')
-> classes that do not have

In [13]:
sound_ins=next(iter(audio_dataset.items()))

In [16]:
sound_ins[1].location_id

'11'

In [17]:
# sound_ins[1].taxo_code
# classA = 1.1.7 #'duck-goose-swan']
# classB = 0.2.0 # other-aircraft
sampleTest= []
y=[]
location_id_info = []
expected_len=10
for sound_ins in audio_dataset.values():
    if sound_ins.taxo_code in ['1.1.10','1.1.7']:
        y.append(sound_ins.taxo_code)
        location_id_info.append(sound_ins.location_id)
        if sound_ins.length<10:
            tile_reps = (expected_len/(sound_ins.length)+1)
            repeated_data = np.tile(sound_ins.data,int(tile_reps))
            repeated_data = repeated_data[:expected_len*sound_ins.sr]
            sampleTest.append(repeated_data)
        else:
            sampleTest.append(sound_ins.data[:expected_len*sound_ins.sr])

len(sampleTest),len(y)

# sampleTest=np.array(sampleTest)

In [18]:
from numpy import argmax
# define input string
# define universe of possible input values
alphabet = ['1.1.10','1.1.7']
# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# integer encode input data
integer_encoded = [char_to_int[char] for char in y]
# print(integer_encoded)
# one hot encode
onehot_encoded = list()
for value in integer_encoded:
	letter = [0 for _ in range(len(alphabet))]
	letter[value] = 1
	onehot_encoded.append(letter)
# print(onehot_encoded)
# invert encoding
inverted = int_to_char[argmax(onehot_encoded[0])]
# print(inverted)
onehot_encoded=np.array(onehot_encoded)

[['49', '48', '19', '16', '22', '37', '29', '25', '31', '27'],
 ['46', '20', '11', '33', '24'],
 ['17', '21', '39', '30', '38', '18', '47', '50', '14']]

In [64]:
X_train,X_test, X_val, y_train, y_test,y_val  = [],[],[],[],[],[]
for sample,taxo,y_val_ins,loc_id in  zip(sampleTest,y,onehot_encoded,location_id_info):
    if loc_id in best_solution_per_taxo_w_loc_name[taxo][0]:
        X_train.append(sample)
        y_train.append(y_val_ins)
    elif loc_id in best_solution_per_taxo_w_loc_name[taxo][1]:
        X_test.append(sample)
        y_test.append(y_val_ins)
    elif loc_id in best_solution_per_taxo_w_loc_name[taxo][2]:
        X_val.append(sample)
        y_val.append(y_val_ins)
    else:
        print('error')


In [113]:
sum(y_test),sum(y_train),sum(y_val)

(tensor([81., 32.]), tensor([265.,  96.]), tensor([85., 30.]))

In [108]:
y_train[21:30]

tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])

In [106]:
onehot_encoded[21:30],y[21:30],location_id_info[21:30]

(array([[0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1]]),
 ['1.1.7',
  '1.1.7',
  '1.1.7',
  '1.1.7',
  '1.1.7',
  '1.1.7',
  '1.1.7',
  '1.1.7',
  '1.1.7'],
 ['15', '15', '15', '15', '15', '15', '15', '15', '15'])

In [101]:
X_train[:10]

tensor([[1702., 1718., 1763.,  ..., 1256., 1268., 1262.],
        [1030., 1052., 1031.,  ..., 1349., 1353., 1306.],
        [1306., 1316., 1358.,  ..., 1159., 1168., 1159.],
        ...,
        [1289., 1260., 1249.,  ..., 1840., 1806., 1836.],
        [1252., 1266., 1271.,  ..., 1759., 1775., 1780.],
        [1071., 1099., 1094.,  ..., 2153., 2156., 2171.]])

In [65]:
X_train,X_test,X_val=np.array(X_train),np.array(X_test),np.array(X_val)
y_train,y_test,y_val=np.array(y_train),np.array(y_test),np.array(y_val)


In [66]:
# X_train, X_test, y_train, y_test = train_test_split(
#                 sampleTest, onehot_encoded, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(
#                 X_train, y_train, test_size=0.25,random_state=42)

In [67]:
X_train,X_test,X_val=torch.from_numpy(X_train).float(),torch.from_numpy(X_test).float(),torch.from_numpy(X_val).float()
y_train,y_test,y_val=torch.from_numpy(y_train).float(),torch.from_numpy(y_test).float(),torch.from_numpy(y_val).float()


In [69]:
X_train.shape,X_test.shape,X_val.shape

(torch.Size([361, 480000]),
 torch.Size([113, 480000]),
 torch.Size([115, 480000]))

In [71]:
y_train.shape,y_test.shape,y_val.shape

(torch.Size([361, 2]), torch.Size([113, 2]), torch.Size([115, 2]))

In [72]:
# # X_train,X_test,X_val,y_train,y_test,y_val = load_raw_data('labelsbyhumanpath','sourcePath')
# X_train,X_test,X_val,y_train,y_test,y_val = mock_raw_data(4,CATEGORY_COUNT)
# X_train,X_test,X_val=torch.from_numpy(X_train).float(),torch.from_numpy(X_test).float(),torch.from_numpy(X_val).float()
# y_train,y_test,y_val=torch.from_numpy(y_train).float(),torch.from_numpy(y_test).float(),torch.from_numpy(y_val).float()

# # labelsbyhumanpath = Path('/scratch/enis/data/nna/labeling/results/')
# # with open(labelsbyhumanpath/"np_array_Ymatrix.npy", 'rb') as f:
# #     y_true = np.load(f)

# Counter(np.argmax(y_train,axis=1).tolist()),Counter(np.argmax(y_val,axis=1).tolist()),Counter(np.argmax(y_test,axis=1).tolist())


In [73]:

# X_train2=np.interp(X_train, (X_train.min(), X_train.max()), (-32768 , 32767))
# torch.from_numpy(X_train2).float()


In [74]:
# y_val

In [75]:
# smaple_index=10
# X_train[smaple_index,:],y_train[smaple_index,]


In [76]:
# X_train[0,:].shape

In [77]:
# multiply(torch.ones((1,2)),torch.ones((1,2)))

In [78]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self, maxMelLen, sampling_rate):
        # sr = 44100 etc
        self.maxMelLen = maxMelLen
        self.sampling_rate = sampling_rate

    def __call__(self, sample):
        x, y = sample
        #https://github.com/PCerles/audio/blob/3803d0b27a4e13efa760227ef6c71d0f3753aa98/test/test_transforms.py#L262
        #librosa defaults
        n_fft = 2048
        hop_length = 512
        power = 2.0
        n_mels = 128
        n_mfcc = 40
        # htk is false in librosa, no setting in torchaudio -?
        # norm is 1 in librosa, no setting in torchaudio -?
        melspect_transform = torchaudio.transforms.MelSpectrogram(sample_rate=self.sampling_rate, window_fn=torch.hann_window,
                                                                  hop_length=hop_length, n_mels=n_mels, n_fft=n_fft)

    
        db_transform = torchaudio.transforms.AmplitudeToDB("power", 80.)
        mel = melspect_transform(x.reshape(-1))
        an_x = db_transform(mel)
        #librosa version
#         mel = librosa.feature.melspectrogram(y=x.reshape(-1),
#                                              sr=self.sampling_rate)
#         an_x = librosa.power_to_db(mel, ref=np.max)
#         an_x = an_x.astype("float32")
#         y = y.astype('float32')
        an_x = an_x[:, :self.maxMelLen]
        # 2-d conv
#         x = an_x.reshape(1, *an_x.shape[:])
        # 1-d conv
        x = an_x.reshape(1, an_x.shape[0]*an_x.shape[1])

        
        return x,y

# #test
# maxMelLen_test = 850
# SAMPLING_RATE_test = 48000
# sample_len_seconds = 10
# # to_tensor works on single sample
# sample_count = 1
# xx_test = torch.ones((sample_count,SAMPLING_RATE_test*sample_len_seconds))
# y_values = torch.ones(sample_count)
# 
# toTensor = ToTensor(maxMelLen_test,SAMPLING_RATE_test)
# x_out,y_out=toTensor((xx_test,y_values))
# x_out.shape,y_out.shape

In [79]:
# X_train,X_test,X_val,y_train,y_test,y_val
X_train[:].shape

torch.Size([361, 480000])

In [80]:
# toTensor = ToTensor(maxMelLen_test,SAMPLING_RATE_test)
# x_out2,y_out=toTensor((X_train[1:2,:],y_train))
# x_out.shape,y_out.shape

In [81]:
# torch.mean(x_out[0]),torch.mean(x_out2[0])
# y_train

In [82]:
# X_train,y_train

In [83]:

# pitch = augmentations.pitch_shift_n_stepsClass(
#     runconfigs.SAMPLING_RATE, config['pitch_shift_n_steps'])
# noise = augmentations.addNoiseClass(config['noise_factor'])
# strech = augmentations.time_stretchClass(runconfigs.SAMPLING_RATE*runconfigs.EXCERPT_LENGTH,
#                                             config['time_stretch_factor'],
#                                             isRandom=True)
# shift = augmentations.shiftClass(config['roll_rate'], isRandom=True)
maxMelLen = 850 # old 850
# toTensor = augmentations.ToTensor(maxMelLen,runconfigs.SAMPLING_RATE)
toTensor = ToTensor(maxMelLen,runconfigs.SAMPLING_RATE)

In [84]:
from ignite.metrics import EpochMetric


def roc_auc_perClass_compute_fn(y_preds, y_targets):
    try:
        from sklearn.metrics import roc_auc_score
    except ImportError:
        raise RuntimeError(
            "This contrib module requires sklearn to be installed.")

    y_true = y_targets.numpy()
    y_pred = y_preds.numpy()
#     print(y_pred,y_true)
#     res = []
#     for y_true_perClass_Index in y_true.shape[1]:
#         res.append(
#             roc_auc_score(y_true[:, y_true_perClass_Index],
#                           y_pred[:, y_true_perClass_Index]))
    res = roc_auc_score(y_true, y_pred, average=None)
    return res


#[docs]
class ROC_AUC_perClass(EpochMetric):
    """Computes Area Under the Receiver Operating Characteristic Curve (ROC AUC)
  accumulating predictions and the ground-truth during an epoch and applying
  `sklearn.metrics.roc_auc_score <http://scikit-learn.org/stable/modules/generated/
  sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score>`_ .

  Args:
      output_transform (callable, optional): a callable that is used to transform the
          :class:`~ignite.engine.engine.Engine`'s ``process_function``'s output into the
          form expected by the metric. This can be useful if, for example, you have a multi-output model and
          you want to compute the metric with respect to one of the outputs.
      check_compute_fn (bool): Optional default False. If True, `roc_curve
          <http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#
          sklearn.metrics.roc_auc_score>`_ is run on the first batch of data to ensure there are
          no issues. User will be warned in case there are any issues computing the function.

  ROC_AUC expects y to be comprised of 0's and 1's. y_pred must either be probability estimates or confidence
  values. To apply an activation to y_pred, use output_transform as shown below:

  .. code-block:: python

      def activated_output_transform(output):
          y_pred, y = output
          y_pred = torch.sigmoid(y_pred)
          return y_pred, y

      roc_auc = ROC_AUC(activated_output_transform)

  """
    def __init__(self,
                 output_transform=lambda x: x,
                 check_compute_fn: bool = False):
#         print(output_transform)
        super(ROC_AUC_perClass,
              self).__init__(roc_auc_perClass_compute_fn,
                             output_transform=output_transform,
                             check_compute_fn=check_compute_fn)


In [85]:

def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    """
  Utility function for computing output of convolutions
  takes a tuple of (h,w) and returns a tuple of (h,w)
  """
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h = floor(((h_w[0] + (2 * pad) - (dilation *
                                      (kernel_size[0] - 1)) - 1) / stride) + 1)
    w = floor(((h_w[1] + (2 * pad) - (dilation *
                                      (kernel_size[1] - 1)) - 1) / stride) + 1)
    return h, w

# mel.shape,an_x.shape,X_train.shape
class testModel(nn.Module):
    '''A simple model for testing by overfitting.
    '''
    def __init__(self, out_channels, h_w, kernel_size, FLAT=False,output_shape=(10,)):
        # h_w: height will be always one since we use 1d convolution 
        super(testModel, self).__init__()
        self.out_channels = out_channels
        #### CONV
        self.conv1 = nn.Conv1d(in_channels=1, # depth of image == depth of filters
                               out_channels=self.out_channels, # number of filters 
                               kernel_size=kernel_size, # size of the filters/kernels
                               padding=1)

        self.conv1_shape = conv_output_shape(h_w, kernel_size=kernel_size, stride=1, pad=1, dilation=1)
        # conv is 1d
        self.conv1_shape = (1,self.conv1_shape[1])
        
        self.fc1 = nn.Linear(self.out_channels * self.conv1_shape[0] *self.conv1_shape[1], 100)  # 100

        self.fc2 = nn.Linear(100,output_shape[0])
        
    def forward(self, x):
#         x = x.reshape(1,)
#         print(x.shape) #  50,1,108800 (850*128)
        x = F.relu(self.conv1(x))
#         x = self.pool(x)
        # x = self.drop(x)
#         print(x.shape)# 58, 2, 108801
#         print(self.conv1_shape)
#         print(x.shape)
        x = x.view(-1, self.out_channels * self.conv1_shape[0] *self.conv1_shape[1])
        # batch_norm is missing
        x = F.relu((self.fc1(x)))
        x = (self.fc2(x))

#         x = self.drop(x)

#         x = self.fc4(x)
#         x = torch.sigmoid(x)
#                 x = F.log_softmax(x,dim=1)
        return x

# test
input_shape=(1,(850*128))
output_shape=(10,)
testModel_ins=testModel(out_channels=2,h_w=input_shape,kernel_size=2,output_shape=output_shape)
# a.conv1.weight
a_out=testModel_ins(torch.ones((3,1,input_shape[1])))

a_out_correct=torch.zeros(a_out.shape)
a_out_correct[0][:]=1
a_out_correct
a_out.detach().numpy()

torch.exp(a_out),a_out

(tensor([[0.9094, 0.9313, 0.9633, 1.0505, 0.9422, 0.9441, 1.1694, 0.8413, 1.0199,
          0.9918],
         [0.9094, 0.9313, 0.9633, 1.0505, 0.9422, 0.9441, 1.1694, 0.8413, 1.0199,
          0.9918],
         [0.9094, 0.9313, 0.9633, 1.0505, 0.9422, 0.9441, 1.1694, 0.8413, 1.0199,
          0.9918]], grad_fn=<ExpBackward>),
 tensor([[-0.0949, -0.0712, -0.0374,  0.0493, -0.0595, -0.0575,  0.1565, -0.1728,
           0.0197, -0.0083],
         [-0.0949, -0.0712, -0.0374,  0.0493, -0.0595, -0.0575,  0.1565, -0.1728,
           0.0197, -0.0083],
         [-0.0949, -0.0712, -0.0374,  0.0493, -0.0595, -0.0575,  0.1565, -0.1728,
           0.0197, -0.0083]], grad_fn=<AddmmBackward>))

In [86]:

# for y_true_perClass_Index in a_out_correct.shape[1]:
#     print(y_true_perClass_Index)

In [87]:
# from ignite.contrib.metrics import ROC_AUC
# from nna.exp.metrics import ROC_AUC_perClass
def activated_output_transform(output):
    y_pred, y = output
#     y_pred = torch.exp(y_pred)
    return y_pred, y

In [88]:
asd=ROC_AUC_perClass(activated_output_transform)
asd.update((a_out,a_out_correct))
asd.compute()

array([0.75, 0.75, 0.25, 0.75, 0.25, 0.75, 0.5 , 0.5 , 0.5 , 0.25])

In [89]:

transformCompose = transforms.Compose([
#     pitch,
#     strech,
#     shift,
#     noise,
    toTensor,
])


sound_datasets = {
    phase: runutils.audioDataset(XY[0], XY[1], transform=transformCompose)
    for phase, XY in
    zip(['train', 'val', 'test'],
        [[X_train, y_train], [X_val, y_val], [X_test, y_test]])
}

dataloaders = {
    x: torch.utils.data.DataLoader(sound_datasets[x], **params)
    for x in ['train', 'val', 'test']
}

# this will change
h_w = [128, 850]
kernel_size = (4, 4)
# if config['CNNLayer_count'] == 1:
#     model = modelArchs.NetCNN1(config['CNN_filters_1'], h_w,
#                                 kernel_size).float().to(device)

# if config['CNNLayer_count'] == 2:
#     model = modelArchs.NetCNN2(config['CNN_filters_1'], config.CNN_filters_2,
#                                 h_w, kernel_size,
#                                 kernel_size).float().to(device)

#simpler model

output_shape=(CATEGORY_COUNT,)
model = testModel(out_channels=2,h_w=(1,h_w[0]*h_w[1]),kernel_size=kernel_size[0]*kernel_size[0],output_shape=output_shape)
model.float().to(device)

# device is defined before

model.float().to(device)  # Move model before creating optimizer
optimizer = torch.optim.AdamW(model.parameters(),
#                                 weight_decay=config['weight_decay'],
                             )



criterion = nn.BCEWithLogitsLoss()
# statHistory={'valLoss':[],'trainLoss':[],'trainAUC':[],'valAUC':[]}

metrics = {
    'loss': Loss(criterion),  # 'accuracy': Accuracy(),
#     'ROC_AUC': ROC_AUC(runutils.activated_output_transform),
    'ROC_AUC': ROC_AUC_perClass(activated_output_transform),

}


In [90]:
model

testModel(
  (conv1): Conv1d(1, 2, kernel_size=(16,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=217574, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=2, bias=True)
)

In [91]:
trainloader = dataloaders['train']

In [92]:
# for epoch in range(20000):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()
#         inputs=inputs.to(device)
#         # forward + backward + optimize
#         outputs = model(inputs)
#         outputs = outputs.to('cpu')
#         loss = criterion(outputs, labels)
# #         print(outputs)
# #         print(labels)
# #         print(loss)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
# #         print(running_loss)
#         if epoch % 20 == 19:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss))
#             running_loss = 0.0

# print('Finished Training')

In [93]:
model.conv1.weight

Parameter containing:
tensor([[[-0.1545,  0.1644, -0.0943,  0.1188, -0.2238, -0.0064,  0.0015,
          -0.1312, -0.1030,  0.1724, -0.0483, -0.1018, -0.1916,  0.1355,
          -0.0890,  0.0563]],

        [[-0.0844, -0.0727, -0.1510, -0.1071,  0.2172,  0.0150,  0.2208,
           0.1825, -0.1383, -0.1413,  0.0207,  0.1968,  0.2195, -0.0570,
          -0.0914,  0.1803]]], device='cuda:1', requires_grad=True)

In [94]:
model.conv1

Conv1d(1, 2, kernel_size=(16,), stride=(1,), padding=(1,))

In [95]:
model.fc1.weight

Parameter containing:
tensor([[-0.0014, -0.0015,  0.0015,  ..., -0.0006, -0.0021, -0.0006],
        [-0.0011, -0.0010, -0.0008,  ..., -0.0007,  0.0016, -0.0018],
        [ 0.0015,  0.0009, -0.0013,  ..., -0.0013,  0.0014, -0.0008],
        ...,
        [ 0.0015, -0.0001, -0.0006,  ..., -0.0002, -0.0006,  0.0004],
        [ 0.0014, -0.0011, -0.0018,  ...,  0.0001, -0.0013, -0.0019],
        [-0.0006, -0.0011, -0.0002,  ...,  0.0001,  0.0021,  0.0015]],
       device='cuda:1', requires_grad=True)

In [96]:

print('ready ?')
runutils.run(model, dataloaders, optimizer, criterion, metrics, device,config, runconfigs.PROJECT_NAME)


ready ?


training/batchloss,229.16756
_step,7
_runtime,154
_timestamp,1610465045
training/loss,596.59767


training/batchloss,▁▇▅▆█▆▂
_step,▁▂▃▅▆▇█
_runtime,▁▁▁▁▁▁█
_timestamp,▁▁▁▁▁▁█
training/loss,▁


wandb: wandb version 0.10.13 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2021-01-12 10:24:16,712 Trainer INFO: Engine run starting with max_epochs=2000.
2021-01-12 10:24:22,460 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-12 10:24:26,818 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:04
2021-01-12 10:24:26,821 Train Evaluator INFO: Engine run complete. Time taken: 00:00:04
2021-01-12 10:24:26,822 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-12 10:24:28,116 Val Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:01
2021-01-12 10:24:30,455 Val Evaluator INFO: Engine run complete. Time taken: 00:00:04
wandb: WARNING Step must only increase in log calls.  Step 1 < 7; dropping {'lr/group_0': 0.001}.
2021-01-12 10:24:30,465 Trainer INFO: Epoch[1] Complete. Time taken: 00:00:14


...

train loss 0.22949515997207726
val loss 0.6197144497995791
validation roc auc [0.55411765 0.49686275] 69


2021-01-12 10:41:51,376 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-12 10:42:13,681 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:22
2021-01-12 10:42:13,684 Train Evaluator INFO: Engine run complete. Time taken: 00:00:22
2021-01-12 10:42:13,685 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-12 10:42:18,247 Val Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:05
2021-01-12 10:42:18,249 Val Evaluator INFO: Engine run complete. Time taken: 00:00:05
wandb: WARNING Step must only increase in log calls.  Step 70 < 490; dropping {'lr/group_0': 0.001}.
2021-01-12 10:42:18,255 Trainer INFO: Epoch[70] Complete. Time taken: 00:00:44


train loss 0.22720928675910443
val loss 0.6139189792715984
validation roc auc [0.55019608 0.49568627] 70


2021-01-12 10:42:41,767 Train Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-12 10:43:00,347 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:19
2021-01-12 10:43:00,350 Train Evaluator INFO: Engine run complete. Time taken: 00:00:19
2021-01-12 10:43:00,351 Val Evaluator INFO: Engine run starting with max_epochs=1.
2021-01-12 10:43:04,703 Val Evaluator ERROR: Engine run is terminating due to exception: .
2021-01-12 10:43:04,706 Trainer ERROR: Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [ ]:
model.fc1.weight

In [ ]:

example_input = (torch.ones((1,1,h_w[0]*h_w[1]))*-32767)
example_input = example_input.float().to(device)
out=model(example_input)
out

activated_output_transform((out,out))

In [ ]:
torch.Tensor([10,10,10])

In [ ]:
sum(model.fc1.weight)

In [ ]:
m = nn.Sigmoid()

loss = nn.BCEWithLogitsLoss()
input = torch.tensor([100.0,100,100], requires_grad=True)
# target = torch.empty(3).random_(2)
target = torch.ones(3)
output = loss((input), target)
input,target,output

In [ ]:
input

In [ ]:
target

In [ ]:
output

In [ ]:
m(input)